Loading datasets

In [83]:
import pandas as pd
# URL of the HappyDB dataset
data_url = "https://raw.githubusercontent.com/megagonlabs/HappyDB/master/happydb/data/cleaned_hm.csv"

# Load the data into a DataFrame named "happydb"
happydb = pd.read_csv(data_url)

In [84]:
# Check the first few rows of the DataFrame
display(happydb.head())

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


In [85]:
import pandas as pd

# GitHub raw file URL
github_url = "https://raw.githubusercontent.com/megagonlabs/HappyDB/master/happydb/data/topic_dict/people-dict.csv"

# Load the data into a DataFrame without considering the first row as the header
people_dict_df = pd.read_csv(github_url, header=None)

# Optionally, you can set the column name if you want
people_dict_df.columns = ["Name"]



In [86]:

# Display the DataFrame
display(people_dict_df)

,Name
0,aunt
1,auntie
2,aunties
3,aunts
4,aunty
...,...
234,act(or|ress)
235,lady
236,teacher
237,celebrity


#Task 1: Identifying Top Three Social Relationships in Happy Moments Using the  People Dictionary

checking the occurences of each social relationship & asssesing top 3 out of the list


In [87]:
# Create an empty dictionary to store counts
social_relationship_counts = {}

# Loop through all social relationships in the "people dictionary"
for relationship in people_dict_df['Name']:
    # Count occurrences of each relationship in "cleaned_hm" column
    count = happydb['cleaned_hm'].str.contains(relationship, case=False).sum()
    social_relationship_counts[relationship] = count

# Sort the dictionary by counts in descending order
sorted_relationships = sorted(social_relationship_counts.items(), key=lambda x: x[1], reverse=True)


<ipython-input-87-d5caff2e4992>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  count = happydb['cleaned_hm'].str.contains(relationship, case=False).sum()


In [88]:

# Print all social relationships and their respective counts
top_three_relationships = sorted_relationships[:3]
for relationship, count in top_three_relationships:
    print(f"{relationship}: {count}")

friend: 13419
men: 7038
son: 6274


### Task 1 Answer: Top 3 social relationships are "friend", "men" and "son" with occurences of 13419, 7038 and 6274 respectively

#Task 2:
Assessing the People Dictionary: Strengths and Weaknesses in Identifying Happy Moment Participants

Based on the dataset, I can divide my insights into strenghts & weaknesses

#Strengths:
1. In my view, I have observed that this dictionary has a diverse spectrum of social relationships which are not only potraying family bonds but also more extended connections. Some specific examples include relationships like pals, neigbour, preschoolers.

2. Also there are so many relationships like bae(before anyone else), gf(girlfriend), ppl(people) which are informal expressions used in daily language



#Weakness:
1. Vagueness of terms: Some terms like 'men' are part of the data but meaning of term as a relationship is quite vague and lacks context

2. Slang: Data might not account for variations in informal context. For example, this data has pals and ppl might be considered different in context. Whereas sometimes, they can mean the same relationship like " they are my pals" or "they are my ppl"


#Establishing Objectives & performing Task 3

There are many objectives considered - lets go step by step

Objective 1 : Remove slang words - some slang words need context to understand relations
For example - They are my 'pals' & They are my 'ppl' may mean the same

and 'bae' and 'love of my life' can also mean the same

removing slang can help us find more actual words in People Dictionary that occur in HappyDB

Task 3 : Objective 1 : Remove slang words - some slang words

In [89]:
#making copy of data
data_text = people_dict_df.copy()

In [90]:
print(data_text)

             Name
0            aunt
1          auntie
2         aunties
3           aunts
4           aunty
..            ...
234  act(or|ress)
235          lady
236       teacher
237     celebrity
238     professor

[239 rows x 1 columns]


In [91]:
# Install spaCy
!pip install -U spacy

# Download and install the 'en_core_web_md' model
!python -m spacy download en_core_web_md

# Import spaCy and load the model
import spacy

# Load the 'en_core_web_md' model
nlp = spacy.load('en_core_web_md')

2023-09-19 22:42:09.303264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


Considering actual words from people dict - removing slang words can help us understand better the relationships without any extra context

In [92]:
import pandas as pd
import nltk
from nltk.corpus import words as nltk_words

# Download the NLTK words dataset if you haven't already
nltk.download('words')

# Get a list of English words from NLTK
nltk_words_set = set(nltk_words.words())

# Create a new list without slang words
filtered_people_dict = [word for word in data_text['Name'] if word in nltk_words_set]

# Create a new DataFrame from the filtered list
filtered_people_df = pd.DataFrame({'word_column': filtered_people_dict})

# Print the new DataFrame
display(filtered_people_df)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


,word_column
0,aunt
1,auntie
2,baby
3,bae
4,boy
...,...
79,stranger
80,lady
81,teacher
82,celebrity


##Objective 1 - Lexicon

Performing lexicon analysis just by removing slang words

In [93]:
# Create an empty dictionary to store counts
non_slang_social_relationship_counts = {}

# Loop through all social relationships in the "people dictionary"
for i in filtered_people_df['word_column']:
    # Count occurrences of each relationship in "cleaned_hm" column
    count_new = happydb['cleaned_hm'].str.contains(i, case=False).sum()
    non_slang_social_relationship_counts[i] = count_new

# Sort the dictionary by counts in descending order
non_slang_sorted_relationships = sorted(non_slang_social_relationship_counts.items(), key=lambda x: x[1], reverse=True)


In [94]:

# Print all social relationships and their respective counts
non_slang_top_three_relationships = non_slang_sorted_relationships[:3]
for i, counts in non_slang_top_three_relationships:
    print(f"{i}: {counts}")

friend: 13419
men: 7038
son: 6274


##Objective 1 Answer -
Even after removing slang words, most common relationships are friend, men and son with counts of 13419, 7038 and 6274 respectively.

Lets try to now look at these 3 most common relationships. friend and son make sense as a relationship but men does not. What is men as a relationship? Is it being a male in general?

##Task 3 : Objective 2
Now try to de generalise most common words

In [95]:
replacement_rules = {'sons': ['son', 'son-in-law', 'step-father', 'stepson'],'friend': ['best friend', 'bestfriend', 'bestfriends'],'men': ['guy', 'guys', 'man']}

In [96]:
def apply_replacements(text):
    for replacement, terms in replacement_rules.items():
        for term in terms:
            text = re.sub(rf'\b{re.escape(term)}\b', replacement, text)
    return text

In [97]:
# Apply the replacement function to the 'value' column
filtered_people_df['word_column'] = filtered_people_df['word_column'].apply(apply_replacements)

In [98]:

# Remove duplicates after consolidation in case any
degeneralise_filtered_people_df = filtered_people_df.drop_duplicates(subset='word_column', keep='first')
display(degeneralise_filtered_people_df)

,word_column
0,aunt
1,auntie
2,baby
3,bae
4,boy
...,...
79,stranger
80,lady
81,teacher
82,celebrity


After removing slang words & degeneralising terms ,we are down to 80 words in people dictionary

##Objective 2 : Lexicon
Lets try doing the analysis after addressing this issue

In [99]:
# Create an empty dictionary to store counts
degeneralise_social_relationship_counts = {}
# Loop through all social relationships in the "people dictionary"
for i in degeneralise_filtered_people_df['word_column']:
    # Count occurrences of each relationship in "cleaned_hm" column
    count_new = happydb['cleaned_hm'].str.contains(i, case=False).sum()
    degeneralise_social_relationship_counts[i] = count_new

# Sort the dictionary by counts in descending order
degeneralise_sorted_relationships = sorted(degeneralise_social_relationship_counts.items(), key=lambda x: x[1], reverse=True)


In [100]:

# Print all social relationships and their respective counts
degeneralise_top_three_relationships = degeneralise_sorted_relationships[:3]
for i, counts in degeneralise_top_three_relationships:
    print(f"{i}: {counts}")

friend: 13419
men: 7038
family: 4222


We can now see change in occurences of social relationships to be friend, men and family as 13419, 7038 and 4222 which is more specific than previous approaches

#Task 4

let us consider top 3 i.e friend, men and son

In [101]:
# Define the most mentioned relationships
most_mentioned_relationships = ["friend", "men", "son"]

In [102]:
import pandas as pd
import re
from collections import Counter
import nltk
nltk.download('stopwords')
# Create a dictionary to store context words and their frequencies
context_words_counter = Counter()

# Get the NLTK stop words list
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [103]:
# Extract context words around each relationship
for relationship in most_mentioned_relationships:
    # Iterate through each row of the dataset
    for idx, row in happydb.iterrows():
        text = row["cleaned_hm"]
        # Find occurrences of the relationship using regex
        occurrences = re.finditer(rf'\b{re.escape(relationship)}\b', text, flags=re.IGNORECASE)
        for match in occurrences:
            # Create a context window of five words before and after the occurrence
            start_idx = max(match.start() - 5, 0)
            end_idx = min(match.end() + 5, len(text))
            context = text[start_idx:end_idx]
            # Tokenize the context into words
            words = context.split()
            # Remove the relationship word and stop words from the context words
            words = [word for word in words if word.lower() != relationship.lower() and word.lower() not in stop_words]
            # Update the context words counter
            context_words_counter.update(words)


In [104]:
# Sort context words by frequency in descending order
sorted_context_words = context_words_counter.most_common()

# Explore patterns in the 100 most frequent context words
top_100_context_words = sorted_context_words[:100]

#Task 4 Answer

In [105]:
# Here is a list of 100 most frequent context words
for word, frequency in top_100_context_words:
    print(f"{word}: {frequency}")

old: 721
best: 692
friend.: 664
h: 558
th: 485
friend's: 371
got: 265
son's: 245
son.: 230
g: 225
n: 225
good: 208
ha: 206
e: 169
came: 164
r: 147
om: 146
gave: 140
lose: 139
told: 130
friend,: 116
f: 116
bi: 111
hool: 110
afte: 106
new: 101
son,: 96
BEST: 94
k: 91
ed: 90
call: 87
BIRT: 87
hood: 86
w: 84
made: 83
,: 76
gest: 76
dest: 75
girl: 74
play: 69
.: 67
BOY: 65
star: 61
MARR: 61
ho: 58
went: 56
ther: 55
LOVE: 55
brou: 52
Aft: 52
sent: 51
p: 49
lear: 49
ng: 46
took: 45
ba: 45
invi: 44
said: 43
rece: 41
boug: 39
last: 38
9: 38
ee: 37
aske: 37
abou: 35
grad: 35
toda: 33
aw: 33
met: 33
marr: 32
c: 32
time: 32
fina: 32
help: 32
dear: 31
l: 31
en: 30
hit: 30
text: 28
lege: 27
H: 27
surp: 26
nd: 24
comp: 24
pass: 24
get: 23
visi: 23
birt: 22
lp: 22
et: 21
ad: 21
ne: 20
move: 20
lder: 20
SCHO: 20
sc: 20
scor: 20
yest: 19
perf: 19
-old: 19


When it comes to finding patterns, some of the words need some context to indicate activities that people do in happy moment.

1. Some words like 'best', 'old', and 'friend' are mentioned but we need some combination of words/sentences to be sure to say its a happy moments

2. Some positive emotions that say this from the output are good-208 times, BEST - 98 times, LOVE - 55 times

3. Some verbs like got- 265 times, came-164, gave - 140 times, made - 83 times, sent - 51 times, help - 32 times



